<a href="https://colab.research.google.com/github/AhanaMittra/Covid-19-Prediction/blob/main/Covid_19_Test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Library Import***

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import math
import time
import operator
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')
%matplotlib inline

# ***Data Preprocessing***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/corona_tested_individuals_ver_006.english.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COVID-19/covid_data.csv')

In [ ]:
data_org.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,corona_result
0,2020-04-30,0,0,0,0,0,1,1,2,0
1,2020-04-30,1,0,0,0,0,1,1,2,0
2,2020-04-30,0,1,0,0,0,1,2,2,0
3,2020-04-30,1,0,0,0,0,1,1,2,0
4,2020-04-30,1,0,0,0,0,1,2,2,0


In [ ]:
data_org = data.reindex(columns=["test_date",	"cough",	"fever",	"sore_throat",	"shortness_of_breath",	"head_ache",	"age_60_and_above",	"gender",	"test_indication", "corona_result"])

In [ ]:
data_org

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,corona_result
0,2020-04-30,0,0,0,0,0,1,1,2,0
1,2020-04-30,1,0,0,0,0,1,1,2,0
2,2020-04-30,0,1,0,0,0,1,2,2,0
3,2020-04-30,1,0,0,0,0,1,1,2,0
4,2020-04-30,1,0,0,0,0,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...
278843,2020-03-11,0,0,0,0,0,1,0,2,0
278844,2020-03-11,0,0,0,0,0,1,0,2,0
278845,2020-03-11,0,0,0,0,0,1,0,1,2
278846,2020-03-11,0,0,0,0,0,1,0,2,1


# Test Indication

In [ ]:
data_org.test_indication.unique()

array([2, 0, 1])

In [ ]:
class_mapping_test_ind = {label:idx for idx, label in enumerate(np.unique(data_org['test_indication']))}

In [ ]:
class_mapping_test_ind

{'Abroad': 0, 'Contact with confirmed': 1, 'Other': 2}

In [ ]:
data_org['test_indication'] = data_org['test_indication'].map(class_mapping_test_ind)

# Gender

In [ ]:
class_mapping_gender = {label:idx for idx, label in enumerate(np.unique(data_org['gender']))}

In [ ]:
class_mapping_gender

{0: 0, 1: 1, 2: 2}

In [ ]:
data_org['gender'] = data_org['gender'].map(class_mapping_gender)

In [ ]:
data_org

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,corona_result
0,2020-04-30,0,0,0,0,0,1,1,2,0
1,2020-04-30,1,0,0,0,0,1,1,2,0
2,2020-04-30,0,1,0,0,0,1,2,2,0
3,2020-04-30,1,0,0,0,0,1,1,2,0
4,2020-04-30,1,0,0,0,0,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...
278843,2020-03-11,0,0,0,0,0,1,0,2,0
278844,2020-03-11,0,0,0,0,0,1,0,2,0
278845,2020-03-11,0,0,0,0,0,1,0,1,2
278846,2020-03-11,0,0,0,0,0,1,0,2,1


# Age 60 and above

In [ ]:
data_org.age_60_and_above.unique()

array([1, 2, 0])

In [ ]:
class_mapping_age_60_and_above = {label:idx for idx, label in enumerate(np.unique(data_org['age_60_and_above']))}
data_org['age_60_and_above'] = data_org['age_60_and_above'].map(class_mapping_age_60_and_above)

In [ ]:
data_org

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,corona_result
0,2020-04-30,0,0,0,0,0,1,1,2,0
1,2020-04-30,1,0,0,0,0,1,1,2,0
2,2020-04-30,0,1,0,0,0,1,2,2,0
3,2020-04-30,1,0,0,0,0,1,1,2,0
4,2020-04-30,1,0,0,0,0,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...
278843,2020-03-11,0,0,0,0,0,1,0,2,0
278844,2020-03-11,0,0,0,0,0,1,0,2,0
278845,2020-03-11,0,0,0,0,0,1,0,1,2
278846,2020-03-11,0,0,0,0,0,1,0,2,1


# Corona Result

In [ ]:
class_mapping_corona_result = {label:idx for idx, label in enumerate(np.unique(data_org['corona_result']))}
data_org['corona_result'] = data_org['corona_result'].map(class_mapping_corona_result)

In [ ]:
data_org

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,corona_result
0,2020-04-30,0,0,0,0,0,1,1,2,0
1,2020-04-30,1,0,0,0,0,1,1,2,0
2,2020-04-30,0,1,0,0,0,1,2,2,0
3,2020-04-30,1,0,0,0,0,1,1,2,0
4,2020-04-30,1,0,0,0,0,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...
278843,2020-03-11,0,0,0,0,0,1,0,2,0
278844,2020-03-11,0,0,0,0,0,1,0,2,0
278845,2020-03-11,0,0,0,0,0,1,0,1,2
278846,2020-03-11,0,0,0,0,0,1,0,2,1


# Cough

In [ ]:
# a = '0'
# print(isinstance(a, int))
# data_org['cough'][1]

data_org['cough'].unique()

array([0, 1, '0', '1', 'None'], dtype=object)

In [ ]:
for i in range(len(data_org.cough)):
  if not isinstance(data_org['cough'][i], str):
    data_org['cough'][i] = str(data_org['cough'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
count = 0
for i in range(len(data_org.cough)):
  if isinstance(data_org['cough'][i], str):
    count += 1

print(count)

278848


In [ ]:
cough_mapping = {
    '0': 0,
    '1': 1,
    'None': 0
}

In [ ]:
data_org['cough'] = data_org['cough'].map(cough_mapping)

In [ ]:
data_org['cough'].unique()

array([0, 1])

# Unique Value Checking

In [ ]:
for i in data_org.columns:
  print(i)
  print(data_org[i].unique())

test_date
['2020-04-30' '2020-04-29' '2020-04-28' '2020-04-27' '2020-04-26'
 '2020-04-25' '2020-04-24' '2020-04-23' '2020-04-22' '2020-04-21'
 '2020-04-20' '2020-04-19' '2020-04-18' '2020-04-17' '2020-04-16'
 '2020-04-15' '2020-04-14' '2020-04-13' '2020-04-12' '2020-04-11'
 '2020-04-10' '2020-04-09' '2020-04-08' '2020-04-07' '2020-04-06'
 '2020-04-05' '2020-04-04' '2020-04-03' '2020-04-02' '2020-04-01'
 '2020-03-31' '2020-03-30' '2020-03-29' '2020-03-28' '2020-03-27'
 '2020-03-26' '2020-03-25' '2020-03-24' '2020-03-23' '2020-03-22'
 '2020-03-21' '2020-03-20' '2020-03-19' '2020-03-18' '2020-03-17'
 '2020-03-16' '2020-03-15' '2020-03-14' '2020-03-13' '2020-03-12'
 '2020-03-11']
cough
[0 1 '0' '1' 'None']
fever
[0 1 '0' '1' 'None']
sore_throat
[0 1 '0' '1' 'None']
shortness_of_breath
[0 1 '0' '1' 'None']
head_ache
[0 1 '0' '1' 'None']
age_60_and_above
[1 2 0]
gender
[1 2 0]
test_indication
[2 0 1]
corona_result
[0 2 1]


# Fever

In [ ]:
for i in range(len(data_org.fever)):
  if not isinstance(data_org['fever'][i], str):
    data_org['fever'][i] = str(data_org['fever'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_org['fever'].unique()

array(['0', '1', 'None'], dtype=object)

In [ ]:
count = 0
for i in range(len(data_org.cough)):
  if isinstance(data_org['fever'][i], str):
    count += 1

print(count)

278848


In [ ]:
fever_mapping = {
    '0': 0,
    '1': 1,
    'None': 0
}

In [ ]:
data_org['fever'] = data_org['fever'].map(fever_mapping)

In [ ]:
data_org['fever'].unique()

array([0, 1])

# Sore Throat

In [ ]:
for i in range(len(data_org.cough)):
  if not isinstance(data_org['sore_throat'][i], str):
    data_org['sore_throat'][i] = str(data_org['sore_throat'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_org['sore_throat'].unique()

array(['0', '1', 'None'], dtype=object)

In [ ]:
count = 0
for i in range(len(data_org.cough)):
  if isinstance(data_org['sore_throat'][i], str):
    count += 1

print(count)

278848


In [ ]:
sore_throat_mapping = {
    '0': 0,
    '1': 1,
    'None': 0
}

In [ ]:
data_org['sore_throat'] = data_org['sore_throat'].map(sore_throat_mapping)

In [ ]:
data_org['sore_throat'].unique()

array([0, 1])

# Shortness of Breath

In [ ]:
for i in range(len(data_org.cough)):
  if not isinstance(data_org['shortness_of_breath'][i], str):
    data_org['shortness_of_breath'][i] = str(data_org['shortness_of_breath'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_org['shortness_of_breath'].unique()

array(['0', '1', 'None'], dtype=object)

In [ ]:
count = 0
for i in range(len(data_org.cough)):
  if isinstance(data_org['shortness_of_breath'][i], str):
    count += 1

print(count)

278848


In [ ]:
shortness_of_breath_mapping = {
    '0': 0,
    '1': 1,
    'None': 0
}

In [ ]:
data_org['shortness_of_breath'] = data_org['shortness_of_breath'].map(shortness_of_breath_mapping)

In [ ]:
data_org['shortness_of_breath'].unique()

array([0, 1])

# Head Ache

In [ ]:
for i in range(len(data_org.cough)):
  if not isinstance(data_org['head_ache'][i], str):
    data_org['head_ache'][i] = str(data_org['head_ache'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_org['head_ache'].unique()

array(['0', '1', 'None'], dtype=object)

In [ ]:
count = 0
for i in range(len(data_org.cough)):
  if isinstance(data_org['head_ache'][i], str):
    count += 1

print(count)

278848


In [ ]:
head_ache_mapping = {
    '0': 0,
    '1': 1,
    'None': 0
}

In [ ]:
data_org['head_ache'] = data_org['head_ache'].map(head_ache_mapping)

In [ ]:
data_org['head_ache'].unique()

array([0, 1])

# Unique Value Checking

In [ ]:
for i in data_org.columns:
  print(i)
  print(data_org[i].unique())

test_date
['2020-04-30' '2020-04-29' '2020-04-28' '2020-04-27' '2020-04-26'
 '2020-04-25' '2020-04-24' '2020-04-23' '2020-04-22' '2020-04-21'
 '2020-04-20' '2020-04-19' '2020-04-18' '2020-04-17' '2020-04-16'
 '2020-04-15' '2020-04-14' '2020-04-13' '2020-04-12' '2020-04-11'
 '2020-04-10' '2020-04-09' '2020-04-08' '2020-04-07' '2020-04-06'
 '2020-04-05' '2020-04-04' '2020-04-03' '2020-04-02' '2020-04-01'
 '2020-03-31' '2020-03-30' '2020-03-29' '2020-03-28' '2020-03-27'
 '2020-03-26' '2020-03-25' '2020-03-24' '2020-03-23' '2020-03-22'
 '2020-03-21' '2020-03-20' '2020-03-19' '2020-03-18' '2020-03-17'
 '2020-03-16' '2020-03-15' '2020-03-14' '2020-03-13' '2020-03-12'
 '2020-03-11']
cough
[0 1]
fever
[0 1]
sore_throat
[0 1]
shortness_of_breath
[0 1]
head_ache
[0 1]
age_60_and_above
[1 2 0]
gender
[1 2 0]
test_indication
[2 0 1]
corona_result
[0 2 1]


In [ ]:
data_org.to_csv(r'/content/drive/MyDrive/Colab Notebooks/COVID-19/covid_data.csv', index=False)

# Partitioning Dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = data_org.iloc[:, 1:9].values, data_org.iloc[:, -1].values

In [ ]:
X.shape

(278848, 8)

In [ ]:
y.shape

(278848,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train

array([[1, 0, 0, ..., 1, 1, 2],
       [0, 0, 0, ..., 1, 1, 2],
       [0, 0, 0, ..., 0, 2, 2],
       ...,
       [0, 0, 0, ..., 1, 1, 2],
       [0, 0, 0, ..., 0, 2, 2],
       [0, 0, 0, ..., 1, 2, 2]])

In [ ]:
X_test

array([[0, 0, 0, ..., 0, 1, 2],
       [0, 0, 0, ..., 0, 1, 2],
       [0, 0, 0, ..., 0, 2, 2],
       ...,
       [1, 0, 0, ..., 1, 1, 2],
       [0, 0, 0, ..., 1, 1, 2],
       [0, 0, 0, ..., 0, 2, 2]])

# ***Model Implementation***

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.predict_proba(X_test)

array([[0.96064955, 0.014679  , 0.02467144],
       [0.96064955, 0.014679  , 0.02467144],
       [0.95783672, 0.01229059, 0.0298727 ],
       ...,
       [0.95825504, 0.01392414, 0.02782082],
       [0.96361171, 0.01426062, 0.02212767],
       [0.95783672, 0.01229059, 0.0298727 ]])

In [ ]:
accuracy_score(y_test, y_pred)

0.9436734205964975

In [ ]:
y_test.shape

(83655,)

In [ ]:
y_pred.shape

(83655,)